In [2]:
# import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline


In [3]:
# read the dataset
df = pd.read_csv('survey_results_public.csv')

df_schema = pd.read_csv('survey_results_schema.csv')


# Exploring the Dataset

In [4]:
df.shape

(51392, 154)

In [5]:
df.head(10)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Student,"Yes, both",New Zealand,"Yes, full-time","Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Disagree,NaN,A bachelor's degree,White or of European descent,Disagree,Agree,Disagree,Agree,NaN,NaN
6,7,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Master's degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...",20 to 99 employees,...,Disagree,Male,A doctoral degree,White or of European descent,Disagree,Agree,Disagree,Agree,NaN,NaN
7,8,Professional developer,"Yes, both",Poland,No,Employed full-time,Master's degree,Computer science or software engineering,All or almost all the time (I'm full-time remote),Fewer than 10 employees,...,Somewhat agree,Male,A master's degree,White or of European descent,Agree,Somewhat agree,Disagree,Agree,NaN,NaN
8,9,Professional developer,"Yes, I program as a hobby",Colombia,"Yes, part-time",Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","5,000 to 9,999 employees",...,Strongly disagree,Male,A bachelor's degree,Hispanic or Latino/Latina,Somewhat agree,Strongly agree,Disagree,Strongly agree,NaN,NaN
9,10,Professional developer,"Yes, I program as a hobby",France,"Yes, full-time","Independent contractor, freelancer, or self-em...",Master's degree,Computer science or software engineering,It's complicated,NaN,...,Disagree,Male,A doctoral degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,NaN,NaN


In [6]:
df.iloc[100]

Respondent                                                                   101
Professional                                              Professional developer
ProgramHobby                                           Yes, I program as a hobby
Country                                                       Russian Federation
University                                                                    No
EmploymentStatus                                             I prefer not to say
FormalEducation                                           I prefer not to answer
MajorUndergrad                                                               NaN
HomeRemote                                                                   NaN
CompanySize                                                                  NaN
CompanyType                                                                  NaN
YearsProgram                                                    20 or more years
YearsCodedJob               

In [7]:
df.InfluenceHardware.head()

0                    NaN
1    No influence at all
2         Some influence
3                    NaN
4                    NaN
Name: InfluenceHardware, dtype: object

In [8]:
df.JobSatisfaction.head()

0    NaN
1    NaN
2    9.0
3    3.0
4    8.0
Name: JobSatisfaction, dtype: float64

In [9]:
df.isnull().sum().sum()

3586951

In [10]:
df_schema.set_index('Column', inplace=True)

In [11]:
df_schema.loc['Professional']

Question    Which of the following best describes you?
Name: Professional, dtype: object

In [12]:
set(df.columns[df.isnull().mean() < 0.25])

{'CareerSatisfaction',
 'ClickyKeys',
 'CompanySize',
 'CompanyType',
 'Country',
 'EmploymentStatus',
 'FormalEducation',
 'HomeRemote',
 'JobSatisfaction',
 'MajorUndergrad',
 'Professional',
 'ProgramHobby',
 'PronounceGIF',
 'Respondent',
 'TabsSpaces',
 'University',
 'YearsCodedJob',
 'YearsProgram'}

In [13]:
df.Country.value_counts()

United States                     11455
India                              5197
United Kingdom                     4395
Germany                            4143
Canada                             2233
France                             1740
Poland                             1290
Australia                           913
Russian Federation                  873
Spain                               864
Netherlands                         855
Italy                               781
Brazil                              777
Sweden                              611
Switzerland                         595
Israel                              575
Romania                             561
Iran                                507
Austria                             477
Pakistan                            454
Czech Republic                      411
Belgium                             404
South Africa                        380
Turkey                              363
Ukraine                             356


In [14]:
df.CompanySize.value_counts()

20 to 99 employees          8587
100 to 499 employees        7274
10,000 or more employees    5680
10 to 19 employees          4103
1,000 to 4,999 employees    3831
Fewer than 10 employees     3807
500 to 999 employees        2486
5,000 to 9,999 employees    1604
I don't know                 869
I prefer not to answer       681
Name: CompanySize, dtype: int64

In [16]:
df.CompanyType.value_counts()

Privately-held limited company, not in startup mode                      16709
Publicly-traded corporation                                               5871
I don't know                                                              3233
Sole proprietorship or partnership, not in startup mode                   2831
Government agency or public school/university                             2451
Venture-funded startup                                                    2387
I prefer not to answer                                                    1816
Pre-series A startup                                                      1288
Non-profit/non-governmental organization or private school/university     1225
State-owned company                                                        670
Something else                                                             342
Name: CompanyType, dtype: int64

In [19]:
df.CollaborateRemote.value_counts()

Somewhat agree       8983
Agree                8784
Disagree             6436
Strongly agree       3519
Strongly disagree    1735
Name: CollaborateRemote, dtype: int64

### Q1. Which developer types does each university produce?
University, CompanyType, DeveloperType, SelfTaughtTypes, EducationTypes
### Q2. How often do university students work from home?
University, HomeRemote
### Q3. How hard is it to collaborate remotly?
CollaborateRemote

# Preproecessing Dataset

which country should we build the company at? Country
what should the company type and size be? CompanySize & CompanyType
Who would we hire? University & ProgramHobby